# Mesh generation for tutorial 06, cases 1a, 2a, 3a, 4a-b, 5, 6, 8

This file generates the mesh which is used in the following examples:
* 1a_poisson
* 2a_advection_diffusion_reaction
* 3a_advection_diffusion_reaction_neumann_control
* 4a_poisson_dirichlet_control
* 4b_poisson_neumann_control_boundary_observation
* 5_stokes
* 6_navier_stokes
* 8_navier_stokes_neumann_control

In [ ]:
import numpy as np
from mpi4py import MPI
from dolfinx import MeshTags, UnitSquareMesh
from dolfinx.io import XDMFFile
from dolfinx.mesh import locate_entities_boundary

In [ ]:
assert MPI.COMM_WORLD.size == 1, "This mesh generation notebook is supposed to be run in serial"

### Create mesh

In [ ]:
mesh = UnitSquareMesh(MPI.COMM_WORLD, 32, 32)

### Create (trivial) subdomains

In [ ]:
num_cells = mesh.topology.index_map(mesh.topology.dim).size_local
subdomains = MeshTags(mesh, mesh.topology.dim,
                      np.arange(0, num_cells), np.zeros((num_cells,), dtype=np.intc))
subdomains.name = "subdomains"

### Create boundaries

In [ ]:
def bottom(x):
    return abs(x[1] - 0.) < np.finfo(float).eps


def left(x):
    return abs(x[0] - 0.) < np.finfo(float).eps


def top(x):
    return abs(x[1] - 1.) < np.finfo(float).eps


def right(x):
    return abs(x[0] - 1.) < np.finfo(float).eps


bottom.id = 1
left.id = 2
top.id = 3
right.id = 4

boundaries_entities = dict()
boundaries_values = dict()
for boundary in (bottom, left, top, right):
    boundaries_entities[boundary.id] = locate_entities_boundary(
        mesh, mesh.topology.dim - 1, boundary)
    boundaries_values[boundary.id] = np.full(
        boundaries_entities[boundary.id].shape, boundary.id, dtype=np.intc)
boundaries = MeshTags(mesh, mesh.topology.dim - 1,
                      np.hstack(list(boundaries_entities.values())),
                      np.hstack(list(boundaries_values.values())))
boundaries.name = "boundaries"

### Save

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "square.xdmf", "w") as output:
    output.write_mesh(mesh)
    output.write_meshtags(subdomains)
    output.write_meshtags(boundaries)